In [1]:
# How to Install OpenCV for Python on Windows?
# https://www.geeksforgeeks.org/how-to-install-opencv-for-python-in-windows/

# !pip install opencv-python

In [2]:
# ! pip install --upgrade pip

In [3]:
# resolve tensorflow issue 
# https://www.askpython.com/python/examples/import-error-no-module-named-tensorflow-found
    
# !pip show tensorflow

In [4]:
# ! pip install tensorflow
# ! pip show tensorflow

In [5]:
# ! pip install imutils

In [6]:
# load_and_proces.py

import pandas as pd
import cv2
import numpy as np

In [7]:
dataset_path = 'fer2013/fer2013/fer2013.csv'
image_size=(48,48)

In [8]:
def load_fer2013():
        data = pd.read_csv(dataset_path)
        pixels = data['pixels'].tolist()
        width, height = 48, 48
        faces = []
        for pixel_sequence in pixels:
            face = [int(pixel) for pixel in pixel_sequence.split(' ')]
            face = np.asarray(face).reshape(width, height)
            face = cv2.resize(face.astype('uint8'),image_size)
            faces.append(face.astype('float32'))
        faces = np.asarray(faces)
        faces = np.expand_dims(faces, -1)
        emotions = pd.get_dummies(data['emotion']).as_matrix()
        return faces, emotions

def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

In [9]:
# real_time_video.py

In [10]:
## [from keras.preprocessing.image import img_to_array]    
from tensorflow.keras.utils import img_to_array
import imutils
import cv2
from keras.models import load_model
import numpy as np
import time

In [11]:
# parameters for loading data and images
detection_model_path = 'haarcascade_files/haarcascade_frontalface_default.xml'
emotion_model_path = 'models/_mini_XCEPTION.102-0.66.hdf5'

In [12]:
# hyper-parameters for bounding boxes shape
# loading models
face_detection = cv2.CascadeClassifier(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)
EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised","neutral"]

In [13]:
# starting video streaming
cv2.namedWindow('your face')
camera = cv2.VideoCapture(0)
while True:
    frame = camera.read()[1]
    #reading the frame
    frame = imutils.resize(frame,width=800)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)
    
    canvas = np.zeros((250, 300, 3), dtype="uint8")
    frameClone = frame.copy()
    if len(faces) > 0:
        faces = sorted(faces, reverse=True,
        key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = faces
                    # Extract the ROI of the face from the grayscale image, resize it to a fixed 28x28 pixels, and then prepare
            # the ROI for classification via the CNN
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (64, 64))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]
    else: continue

 
    for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                # construct the label text
                text = "{}: {:.2f}%".format(emotion, prob * 100)

                # draw the label + probability bar on the canvas
                # emoji_face = feelings_faces[np.argmax(preds)]

                
                w = int(prob * 300)
                cv2.rectangle(canvas, (7, (i * 35) + 5), (w, (i * 35) + 35), (0, 0, 255), -1)
                cv2.putText(canvas, 
                            text, 
                            (10, (i * 35) + 23),
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            0.45,
                            (255, 255, 255), 
                            2)
                cv2.putText(frameClone, label, (fX, fY - 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.00, (0, 0, 255), 2)
                cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
                              (0, 0, 255), 2)
#    for c in range(0, 3):
#        frame[200:320, 10:130, c] = emoji_face[:, :, c] * \
#        (emoji_face[:, :, 3] / 255.0) + frame[200:320,
#        10:130, c] * (1.0 - emoji_face[:, :, 3] / 255.0)


    cv2.imshow('your face', frameClone)
    ## cv2.imshow("Probabilities", canvas)
    
    timestr = time.strftime("%Y%m%d-%H%M%S")
    fileName1 = "output/"+timestr+"_"+label+".jpg"
    print(fileName1)
    cv2.imwrite(filename=fileName1, img=frameClone)
    
    
    print(label)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

1/1 [==============================] - 1s 641ms/step
output/20231104-223955_happy.jpg
happy
1/1 [==============================] - 0s 39ms/step
output/20231104-223955_happy.jpg
happy
1/1 [==============================] - 0s 20ms/step
output/20231104-223955_neutral.jpg
neutral
1/1 [==============================] - 0s 36ms/step
output/20231104-223955_happy.jpg
happy
1/1 [==============================] - 0s 31ms/step
output/20231104-223956_happy.jpg
happy
1/1 [==============================] - 0s 37ms/step
output/20231104-223956_happy.jpg
happy
1/1 [==============================] - 0s 26ms/step
output/20231104-223956_sad.jpg
sad
1/1 [==============================] - 0s 27ms/step
output/20231104-223956_happy.jpg
happy
1/1 [==============================] - 0s 20ms/step
output/20231104-223956_happy.jpg
happy
1/1 [==============================] - 0s 32ms/step
output/20231104-223956_happy.jpg
happy
1/1 [==============================] - 0s 20ms/step
output/20231104-223956_sad.jpg
sad
1/